In [1]:
from __future__ import print_function, division
import torch
from torchvision.datasets.folder import DatasetFolder
from torchvision.datasets.folder import default_loader
import torch.nn as nn
import os
from sklearn.metrics import confusion_matrix
from torchvision.transforms.functional import normalize
from torchvision.transforms import transforms
from torchvision.transforms.functional import normalize
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Normalize
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
from PIL import Image
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import drive

######################################### Barış Kaplan (0069054) ##########################################################################
######################################### Canberk Eker (0060233) ##########################################################################



drive.mount('/content/drive')
data_directory = '/content/drive/MyDrive/dataset/'
os.chdir(data_directory)


print("---------------------------------BEGINNING OF TRAINING THE CNN MODEL----------------------------------------------------------")
print()
transforms_of_datas = {
    'training': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ############### This applies input normalization #############
    ]),
    'validation': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ############### This applies input normalization #############
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ############### This applies input normalization #############
    ]),
}

class MyCustomDefinedDataset(Dataset):
    def __init__(self, root_dir, label_file, transform=None):
        self.root_dir = root_dir
        self.label_file = label_file
        self.transform = transform
        lst_of_files = list()
        for file in os.listdir(root_dir):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                lst_of_files.append(file)
        self.file_list = lst_of_files
        self.labels = self.load_labels()

    def __getitem__(self, index):
        file_name = self.file_list[index]
        image_path = os.path.join(self.root_dir, file_name)
        try:
            image = Image.open(image_path).convert('RGB')
        except (FileNotFoundError, OSError):
            ####### Skip the invalid image files ########
            return self.__getitem__((index + 1) % len(self))

        if self.transform is not None:
            image = self.transform(image)

        label = self.labels[index]
        return image, label

    def load_labels(self):
        label_path = os.path.join(self.root_dir, '..', self.label_file)
        with open(label_path, 'r') as f:
            labels = [int(line.strip()) - 1 for line in f]  
        return list(labels)

    def __len__(self):
        return len(self.file_list)


full_training_dataset = MyCustomDefinedDataset(os.path.join(data_directory, 'training'), 'training_labels.txt',
                                      transform=transforms_of_datas['training'])

### Split the full training dataset into training and validation portions ###
train_size = int(0.70 * len(full_training_dataset))  ########### assigning 70 percent of full training dataset as training portion ###############
valid_size = len(full_training_dataset) - train_size  ########### assigning 30 percent of full training dataset as validation portion ###############
train_dataset, valid_dataset = torch.utils.data.random_split(full_training_dataset, [train_size, valid_size]) ####### train and validation dataset splitting ######

image_datasets = {
    'training': train_dataset,
    'validation': valid_dataset,
    'test': MyCustomDefinedDataset(os.path.join(data_directory, 'test'), 'test_labels.txt',
                                   transform=transforms_of_datas['test'])
}


#####this function assigns equal weights to each image in each dataset type#####
def do_not_handle_class_imbalance_and_assign_equal_weights(imgs, total_class_number):
    wgh_values_not_balanced = {}
    data_types_lst = ['training', 'validation', 'test']
    for data_typ in data_types_lst:
        length_of_img_lst = len(imgs[data_typ])
        values_of_weights = [1.0] * length_of_img_lst  ####### Assigning equal weight of 1.0 to all images ############
        wgh_values_not_balanced[data_typ] = values_of_weights
    print(wgh_values_not_balanced)
    return wgh_values_not_balanced


#####this function handles the class imbalance problem by assigning lower weights to overrepresented images and higher weights to underrepresented ones#####
def handle_class_imbalance_and_assign_weights(imgs, total_class_number):
    lst_cntr = []
    for w in range(0, total_class_number):
        lst_cntr.append(0)
    wgh_values_balanced = {}
    data_types_lst = ['training', 'validation','test']
    for data_typ in data_types_lst:
        for img_name in imgs[data_typ]:
          value_of_label = img_name[1]
          lst_cntr[value_of_label] = lst_cntr[value_of_label] + 1
        weight_vals_of_cls = list()
        for d in range(0, total_class_number):
          count_of_all = sum(lst_cntr)
          weight_val = float(count_of_all) / lst_cntr[d]
          weight_vals_of_cls.append(weight_val)
        values_of_weights = []
        length_of_img_lst = len(imgs[data_typ])
        for q in range(0, length_of_img_lst):
          values_of_weights.append(0)
        for loc, val in enumerate(imgs[data_typ]):
          label_value = val[1]
          values_of_weights[loc] = weight_vals_of_cls[label_value] ############### weight value assignment to each image ###########
        wgh_values_balanced[data_typ] = values_of_weights
    return wgh_values_balanced


weight_values_with_balance = handle_class_imbalance_and_assign_weights(image_datasets, 3)
all_data_types = ['training', 'validation', 'test']
weighted_random_sampler = {k: torch.utils.data.sampler.WeightedRandomSampler(weight_values_with_balance[k], len(weight_values_with_balance[k])) for k in all_data_types}
vars_dataload = {l: torch.utils.data.DataLoader(image_datasets[l],batch_size = 4, shuffle = True, num_workers = 4) for l in all_data_types}
device_in_torch = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
convolutional_model = models.alexnet(pretrained = True)
for param in convolutional_model.parameters():
  param.requires_grad = False


size_of_input = convolutional_model.classifier[6].in_features
convolutional_model.classifier[6] = nn.Linear(size_of_input, 3)
convolutional_model = convolutional_model.to(device_in_torch) 
criterion = nn.CrossEntropyLoss() ####### cross entropy loss function definition #########
convolutional_model_optimizer = optim.SGD(convolutional_model.parameters(), lr=0.0015, momentum=0.75)
exp_learning_rate_scheduler = lr_scheduler.StepLR(convolutional_model_optimizer, step_size=4, gamma=0.10)
convolutional_model.eval


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_correct_guesses = 0
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in vars_dataload['training']:
            inputs = inputs.to(device_in_torch)
            labels = labels.to(device_in_torch)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        model.eval()
        number_of_corrects = 0
        for inputs, labels in vars_dataload['validation']:
            inputs = inputs.to(device_in_torch)
            labels = labels.to(device_in_torch)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                pred_equal_label_sum = torch.sum(preds == labels.data)
                number_of_corrects = number_of_corrects + pred_equal_label_sum
            if number_of_corrects > best_correct_guesses:
                best_correct_guesses = number_of_corrects
                best_model_wts = copy.deepcopy(model.state_dict())
        scheduler.step()
    model.load_state_dict(best_model_wts)
    return model

trained_convolutional_model = train_model(convolutional_model, criterion, convolutional_model_optimizer,exp_learning_rate_scheduler, num_epochs=100)
data_types_lst = ['training', 'validation', 'test']
c1 = 0
i1 = 0
c2 = 0
i2 = 0
c3 = 0
i3 = 0
with torch.no_grad():
  for data_type in data_types_lst:
    for images_inputted, classes_of_images in vars_dataload[data_type]:
      inputted_to_device = images_inputted.to(device_in_torch)
      corresponding_classes_to_device = classes_of_images.to(device_in_torch)
      outcome_values = trained_convolutional_model(inputted_to_device)
      test_values, values_of_predictions = torch.max(outcome_values, 1)
      for lbl_of_class, value_of_prediction in zip(corresponding_classes_to_device, values_of_predictions):
        if int(lbl_of_class) == 0:
          if int(value_of_prediction) == 0:
            c1 = c1 + 1
          else:
            i1 = i1 + 1
        if int(lbl_of_class) == 1:
          if int(value_of_prediction) == 1:
            c2 = c2 + 1
          else:
            i2 = i2 + 1
        if int(lbl_of_class) == 2:
          if int(value_of_prediction) == 2:
            c3 = c3 + 1
          else:
            i3 = i3 + 1

    print()
    print("-------------------------------------------------------------------------------------------")
    print("For the " + str(data_type) + " dataset: ")
    print()
    value_of_ovr_acc = (c1 + c2 + c3) / (c1 + i1 + c2 + i2 + c3 + i3)
    print()
    print("The value of overall accuracy is the following: " + str(value_of_ovr_acc) + "")
    print()
    values_of_class_accuracies = [c1/(c1+i1), c2/(c2+i2), c3/(c3+i3)]
    print("The values of class accuracies are the following: " + str(values_of_class_accuracies) + "")
    print()
    for i, class_accuracy in enumerate(values_of_class_accuracies):
        print("The class accuracy for the Class " + str(i + 1) + " is the following: " + str(class_accuracy) + "")
    print("-------------------------------------------------------------------------------------------")


print()
print("-----------------------------------END OF TRAINING THE CNN MODEL----------------------------------------------------------------")

Mounted at /content/drive
---------------------------------BEGINNING OF TRAINING THE CNN MODEL----------------------------------------------------------



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also


-------------------------------------------------------------------------------------------
For the training dataset: 


The value of overall accuracy is the following: 0.9615384615384616

The values of class accuracies are the following: [0.9487179487179487, 0.9838709677419355, 0.9310344827586207]

The class accuracy for the Class 1 is the following: 0.9487179487179487
The class accuracy for the Class 2 is the following: 0.9838709677419355
The class accuracy for the Class 3 is the following: 0.9310344827586207
-------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------
For the validation dataset: 


The value of overall accuracy is the following: 0.8763440860215054

The values of class accuracies are the following: [0.7666666666666667, 0.9659090909090909, 0.8421052631578947]

The class accuracy for the Class 1 is the following: 0.7666666666666667
The class accu